In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [ ]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

In [3]:
# Preprocess the data 
## Drop irrelevant column
# Name , Row Number has nothing to do with model
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
#axis = 1 means column and axis = 0 means row
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Encode categorical Value
# for e.g. male and female cannot be understood by the model so it convert it into 0 and 1

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [14]:
# for countries like Spain , Germany , France
# we use One hot encoding 
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
#data[['Geography']] gives 2D dataframe
#data['Geography']  gives 1D dataframe which is rejected by OneHotEncoder

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [15]:
one_hot_encoder_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [17]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [18]:
# Creating dataframe for above Encoded categories
geo_encoder_df = pd.DataFrame(geo_encoder,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [20]:
# Substituting Geography name in data with Encoded Geo values
data = pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [21]:
# Save conders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

In [33]:
# Dividing the dataset into dependent and independent set
# Here  Exit will be dependent(output) rest all independent
X = data.drop('Exited',axis=1)
Y = data['Exited']

# Split the data in training and testing sets

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)


# Scaler these features
# Scaling helps to reduce large difference 

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# train data are fit and transorm
# test data transformed based on previously fitted data

In [23]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [34]:
# Dumping Scaler into pickle file after fit_transform
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

### ANN Implementation

In [36]:
import tensorflow as tf
from tensorflow.keras.models import Sequential #for seq model
from tensorflow.keras.layers import Dense # for nodes (Hidden Layer)
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [41]:
(X_train.shape[1],)
#Input nodes of ANN seq  model

(12,)

In [42]:
## Building ANN Model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #Hidden Layer 1
    Dense(32,activation='relu'), # HL2
    Dense(1,activation='sigmoid') ## Output Layer
    # Sigmoid used for binary classification
])

c:\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [43]:
model.summary()
# layer 1  has 832 param (12 x 64 (weights)+ 64(biases))
# layer 2 has 2080 param (62 x 32 + 32)
# layer 3 has 33 param (32 x 1 + 1)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [44]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
# Optimizer adjust weights to reduce loss function
# Here learning rate shows by what number weights will be adjusted
# If if it too high , the model may overshoot the expected value for minimum loss function
# If it is too low , the process will be long
loss = tf.keras.losses.BinaryCrossentropy()
# Loss function shows how far results are from expected output
# Lower the value of loss function higher is the accuracy
loss

In [45]:
## compile model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])
# This argument specifies the metrics that you want to track during training and evaluation.
# Metrics provide insights into how well the model is performing beyond just the loss.

In [46]:
# Setup tensorboard
# This call back is used at time of training to create req info for analysis purpose
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq=1)
# Setting it to 1 means histograms will be logged at the end of each epoch.
# Epoch is one forwad and backward propgation

In [47]:
# Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
#This is a Keras callback that allows you to stop training when a monitored quantity (such as validation loss) has stopped improving. 
# It helps prevent overfitting by stopping training at the point when the model's performance on the validation set begins to decline.
# restore best weight gives out the best final weights from all the epochs

In [48]:
# Training the model
history = model.fit(
    X_train,Y_train,validation_data = (X_test,Y_test),epochs = 100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8173 - loss: 0.4316 - val_accuracy: 0.8375 - val_loss: 0.3867
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8464 - loss: 0.3643 - val_accuracy: 0.8575 - val_loss: 0.3528
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8610 - loss: 0.3481 - val_accuracy: 0.8480 - val_loss: 0.3499
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8572 - loss: 0.3409 - val_accuracy: 0.8595 - val_loss: 0.3460
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8672 - loss: 0.3233 - val_accuracy: 0.8575 - val_loss: 0.3348
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8601 - loss: 0.3362 - val_accuracy: 0.8585 - val_loss: 0.3415
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8685 - loss: 0.3243 - val_accuracy: 0.8615 - val_loss: 0.3362
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8702 - loss: 0.3145 - val_accu

In [50]:
model.save('model.keras')

In [53]:
# Load tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [55]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 23312), started 0:05:08 ago. (Use '!kill 23312' to kill it.)